<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex4_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4


Per group create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)

1.   Change the data source from "frankenstein.txt" to "dracula.txt", 
2.   Train the network for 10 epochs and test it on the 3 example sentences
3.   Write down 3 more example sentences and test the model on these as well
4.   Train the LSTM for 50 more epochs and run the completion again. What do you observe?
4.   Add another LSTM layer to our RNN model
5.   Train the 2-layer LSTM for 10 epochs and see how the sentence completion has changed
6.   Change the "dracula.txt" to "trump.txt", which contains Tweets from Donald Trump from 2019 and 2020.
7.   Train a single-layer LSTM on the Tweet's from Trump. If you run out of memory while preprocessing the data, change ```step = 3``` to ```step = 10```.
8.   Run the completion on the six sentences from before again. What do you observe?



**Question**
The models make very few spelling errors and few grammar error, but the content is mostly nonsense. Why do you think are the error types so different?

We will meet in the general channel at 12:10!

In [ ]:
! nvidia-smi

Fri Oct 30 10:13:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import numpy as np

# Downloading the data

In [ ]:
filename = "dracula.txt"
path = tf.keras.utils.get_file(
    filename, origin=f"https://pub.ist.ac.at/~mlechner/datasets/{filename}"
)
with open(path, encoding="utf-8") as f:
    text = f.read().lower()           # Make lowercase 
text = text.replace("\n", " ")        # Remove line-breaks & newlines
print("Corpus length:", len(text))

860160/857524 [==============================] - 1s 1us/step
Corpus length: 842159


In [ ]:
# Print the first 100 characters of the file we downloaded
print("First 100 characters: ",text[:100])
print("Last 100 characters : ",text[-100:])

First 100 characters:  dracula, by bram stoker  chapter i  jonathan harker's journal  (_kept in shorthand._)   _3 may. bist
Last 100 characters :  will understand how some men so loved her, that they did dare much for her sake."  jonathan harker. 


# Define input features and output labels

In [ ]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

Total chars: 64
Number of sequences: 280703


In [ ]:
print("First sentence:")
print(sentences[0])
print("First label: ")
print(next_chars[0])

First sentence:
dracula, by bram stoker  chapter i  jonathan harke
First label: 
r


# Transforming text into numerical data

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros(len(sentences), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]

In [ ]:
# First training samples
print(x[0])
print("label: ")
print(y[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
label: 
42


# Splitting training and validation data

In [ ]:
# We can define our training and validation dataset as simple numpy arrays
split_index = int(0.2*x.shape[0])
valid_x, valid_y = x[-split_index:],y[-split_index:]
train_x, train_y = x[:-split_index],y[:-split_index]
print("valid_x.shape",valid_x.shape)
print("valid_y.shape",valid_y.shape)
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)

valid_x.shape (56140, 50, 64)
valid_y.shape (56140,)
train_x.shape (224563, 50, 64)
train_y.shape (224563,)


# Defining our Recurrent Neural Network

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(maxlen, len(chars))),
        tf.keras.layers.LSTM(32,return_sequences=True),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["sparse_categorical_accuracy"])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 50, 32)            12416     
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
Total params: 22,848
Trainable params: 22,848
Non-trainable params: 0
_________________________________________________________________


# Training

In [ ]:
training_log = model.fit(train_x, train_y, batch_size=64, epochs=10, validation_data=(valid_x,valid_y))

Epoch 1/10
3509/3509 [==============================] - 28s 8ms/step - loss: 28.1523 - sparse_categorical_accuracy: 0.1296 - val_loss: 18.4078 - val_sparse_categorical_accuracy: 0.1338
Epoch 2/10
3509/3509 [==============================] - 28s 8ms/step - loss: 12.7606 - sparse_categorical_accuracy: 0.1979 - val_loss: 11.3592 - val_sparse_categorical_accuracy: 0.2374
Epoch 3/10
3509/3509 [==============================] - 27s 8ms/step - loss: 7.6625 - sparse_categorical_accuracy: 0.2393 - val_loss: 5.5947 - val_sparse_categorical_accuracy: 0.2330
Epoch 4/10
3509/3509 [==============================] - 28s 8ms/step - loss: 3.9261 - sparse_categorical_accuracy: 0.2833 - val_loss: 4.8535 - val_sparse_categorical_accuracy: 0.2841
Epoch 5/10
3509/3509 [==============================] - 28s 8ms/step - loss: 3.0826 - sparse_categorical_accuracy: 0.3259 - val_loss: 2.1112 - val_sparse_categorical_accuracy: 0.4129
Epoch 6/10
3509/3509 [==============================] - 29s 8ms/step - loss: 1.98

# Let's try out our model

In [ ]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
def complete_sentence(sentence):
  if len(sentence) < maxlen:
    raise ValueError(f"Sentence must have at least {maxlen} characters, but provided sentence had only {sentence}")
  if len(sentence) > maxlen:
    sentence = sentence[-maxlen:]
  generated = ""
  print(f"Continuing sentence '{sentence}' ...")

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = np.argmax(preds) # next_index = sample(preds)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  print("... ", generated)
  print()

In [ ]:
complete_sentence("this is an example to show how the machine would continue")

Continuing sentence ' an example to show how the machine would continue' ...
...   to the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stoust the stous



In [ ]:

complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play cricket with dhoni because i always liked him as a cricketer")
complete_sentence("I want to play bawling with cummins because i always liked him as a bowling")
complete_sentence("Tvbhjjbjbjbbab nv anhvhaghchvdejbbdmn nc n  nac xn jcjb jcejb  eacn cc  acc")
complete_sentence("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj")



Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...   of the an ording with a she will have made the sight. the solerable of the death i saw the horrid strong and the short and the corner that so that i did startle of to see it with it. i must be the mi

Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...  s room and the night, but one of the start are according the sight and the horrid or great with the night. he was the words of his stative fellown and a dear, and he was enterrast on the lething that 

Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...   the other and seemed to the kind. i keep it is one time and that i could see her or working his death. the a sperhode of the softice dided and he was when stent that i sutterened and go on, the words

Continuing sentence 'th dhoni because i always liked him as a cricketer' ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...  ich that i have had so crednt the sound, of the body and darked that the some one since in the hing in the door still a orounient of look which had a seeming and avolding the sight; and then van her, 

Continuing sentence 'th cummins because i always liked him as a bowling' ...
...   of the engle, and like the death--he all it all the patient of polling at one best.  "we not be indeet we will let me affard. i thought had have looked the despolling, as the street of a man as it wa

Continuing sentence 'hchvdejbbdmn nc n  nac xn jcjb jcejb  eacn cc  acc' ...
...  ation. they can see the hesse did"able of more one as i had in be despaige. and i shall one for her afternoon bent thing and seeming the morning of her door of the time. the patow will comes had been 

Continuing sentence 'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj' ...
...  eu"tatch his sighters. i shall such an about the sortion of the since.         *       *       *       *       *       *       *       *       

In [ ]:
complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play football with ronaldo because i always liked him as a footballer")
complete_sentence("I want to play cricket with dhoni because i always liked him as a cricketer")
complete_sentence("I want to play bawling with cummins because i always liked him as a bowling")
complete_sentence("Tvbhjjbjbjbbab nv anhvhaghchvdejbbdmn nc n  nac xn jcjb jcejb  eacn cc  acc")
complete_sentence("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj")

Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...   even it of the stath my to the sable and the count of him and all you with the ceter had count, you at as the wite the state as the hid with has me, and the to the count one, and i was we stand sting

Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...   to accripe the see had me be was my to know it i at one, but as he to be has and momere may of my to firth her as to her cloft the astant bes she of the be prest you one a tomy with the whily, and th

Continuing sentence 'ronaldo because i always liked him as a footballer' ...
...  s that i was and to and manctes to one stell of and the lother, and treat ran the biltoth he to the some the seemed that he can and hand the that the baary keep that the pire the rand the place how th

Continuing sentence 'th dhoni because i always liked him as a cricketer' ...
...  . there rear the was and in the stair was to was that i stand was 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...  s to her in full from of it and the to at the what only to had adtienst of sised in was the lust was and in the datters her the minh a was the deal went were blead, i that draraked at she that and we 

Continuing sentence 'hchvdejbbdmn nc n  nac xn jcjb jcejb  eacn cc  acc' ...
...  on of he stole--when the boback in he her the the to pince the tapated of the sole to the drein i the seemed which i carn that is only stast the stime of in shard the stering the wurt pient her to was

Continuing sentence 'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj' ...
...  '; the would and so the was could for of the fall his redvered her to before the wre were me to were a sittren to the pawt i am so seemed her the sume mine, and the bodder. the sear far had and fall t



In [ ]:
### TODO: Add 3 more sentences to complete. Think about what types of sentences would be interesting to test

# Biased data -> biased model

In [ ]:
complete_sentence("    was the thiefs answer, to which the man replied ")
complete_sentence("    was the thiefs answer, to which the woman replied ")

Continuing sentence '  was the thiefs answer, to which the man replied ' ...
...  to the best, and now the or few sound to the escian of the stile plowhiness, and his like, the sstarting and had the count. i have san interated as he had site that it was form one of charner. she had

Continuing sentence 'was the thiefs answer, to which the woman replied ' ...
...  to me, and the his more in the bidd a thing of the patient. i decrided the door and we sat and sitting that i have treeted to be all the hear, and all dilids and he had been remore a short and which t

